In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from sklearn.linear_model import LinearRegression

from statsmodels.regression.quantile_regression import QuantReg
import statsmodels.formula.api as smf
from scipy.stats import norm
import linear_utils

### Initialize the data from deaths.csv into a dataframe.

In [45]:
url = 'https://raw.githubusercontent.com/mnuzen/156b-learning-support/master/data/us/covid/deaths.csv'
df = pd.read_csv(url, header=0)

### Process data, initializing cumulative cases.

In [50]:
cum_cases = df.iloc[:, 4:]
countyFIPS = df.iloc[:, 0].to_numpy()
cum_cases = cum_cases.to_numpy()

all_zeros = [0 for i in range(14)]
PREDS = []
reg = LinearRegression()
for i in range(len(cum_cases)):
    test = cum_cases[i, -14:]
    train = cum_cases[i, :-14]
    if train[-1] == 0: # no training
        PREDS.append(all_zeros)
    else:
        j = 0
        while train[j] == 0:
            j+=1
        j-=1
        y = train[j:]
        x = np.array(range(len(y))).reshape((len(y), 1))
        x_pred = np.array(range(len(y), len(y)+14)).reshape((14,1))
        reg.fit(x, y)
        pred = reg.predict(x_pred)
        PREDS.append(pred)

PREDS = np.array(PREDS)

### Assuming normal distribution, predict future data. Ensure final prediction shapes and date formatting are correct.

In [52]:
FINAL_PRED = linear_utils.linear(PREDS)
FINAL_PRED = np.array(FINAL_PRED)

(44730, 10)
['2020-05-17-17203' '0.0' '0.0' '0.0' '0.0' '0.007936507936507908'
 '0.3283243993226579' '0.6487122907088081' '0.9691001820949583'
 '1.2894880734811083']
(3195, 14)


### Return data in CSV formatting.

In [ ]:
df = pd.DataFrame(data={"id": FINAL_PRED[:, 0],
                        "10": FINAL_PRED[:, 1],
                        "20": FINAL_PRED[:, 2],
                        "30": FINAL_PRED[:, 3],
                        "40": FINAL_PRED[:, 4],
                        "50": FINAL_PRED[:, 5],
                        "60": FINAL_PRED[:, 6],
                        "70": FINAL_PRED[:, 7],
                        "80": FINAL_PRED[:, 8],
                        "90": FINAL_PRED[:, 9]})

df.to_csv("./submission_linreg_daily.csv", sep=',',index=False)

Our code takes about five minutes to run from beginning to end, relying mostly on a Python function (linear) implemented in a Python file named linear_utils, which we have imported in the beginning of the notebook. 